# Dataset

https://www.kaggle.com/datasets/jensenbaxter/10dataset-text-document-classification

In [1]:
import zipfile
with zipfile.ZipFile('/content/Dataset_text.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Dataset_text/')

Lectura por lotes desde archivo

In [2]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "/content/Dataset_text",
    batch_size=batch_size,
    validation_split=0.2,
    label_mode="categorical",
    seed = 2023,
    subset="training",
)

Found 1000 files belonging to 10 classes.
Using 800 files for training.


In [3]:
val_ds = keras.utils.text_dataset_from_directory(
    "/content/Dataset_text",
    batch_size=batch_size,
    validation_split=0.2,
    label_mode="categorical",
    seed = 2023,
    subset="validation",
)

Found 1000 files belonging to 10 classes.
Using 200 files for validation.


Vectorización de datos

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TextVectorization,GlobalMaxPooling1D, Dropout, MultiHeadAttention, Dense, LayerNormalization, Embedding

max_length = 600
max_tokens = 20000
text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,    # Limitar la entrada a 600 palabras
)

# Extraer solo el texto (features) de los datos de entrada para calcular el vocabulario
text_only_train_ds = train_ds.map(lambda x, y: x)

text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

# Transformer encoder

In [5]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [Dense(dense_dim, activation="relu"),
             Dense(embed_dim),]
        )
        self.layernorm_1 = LayerNormalization()
        self.layernorm_2 = LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

# Positional embedding

In [8]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = Embedding(                  # Capa embedding para los tokens
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = Embedding(                   # Capa embedding para la posición. Requiere conocer previamente la longitud de la secuencia
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        # Wrap tf.math.not_equal in a Lambda layer
        return layers.Lambda(lambda x: tf.math.not_equal(x, 0))(inputs)

    def get_config(self):                             # Serialización
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

# Modelo (Self attention)

In [9]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = tf.keras.Input(shape=(None,), dtype="int64")

x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = GlobalMaxPooling1D()(x)
x = Dropout(0.5)(x)
outputs = Dense(10, activation="softmax")(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:915: UserWarning: Layer 'transformer_encoder' (of type TransformerEncoder) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ positional_embedding_1    │ (None, None, 256)      │      5,273,600 │ input_layer_1[0][0]    │
│ (PositionalEmbedding)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda (Lambda)           │ (None, None)           │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder       │ (None, None, 256)      │        543,776 │ positional_embedding_… │
│ (TransformerEncoder)      │                        │                │ lambda[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d      │ (None, 256)            │              0 │ transformer_encoder[0… │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 256)            │              0 │ global_max_pooling1d[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 10)             │          2,570 │ dropout_1[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 5,819,946 (22.20 MB)

 Trainable params: 5,819,946 (22.20 MB)

 Non-trainable params: 0 (0.00 B)

# Entrenamiento

In [10]:
"""
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_encoder.h5",
                                    save_best_only=True)
]
"""

model.fit(int_train_ds,
          validation_data=int_val_ds,
          epochs=20,
          #callbacks=callbacks
          )

"""
model = keras.models.load_model(
    "transformer_encoder.h5",
    custom_objects={"TransformerEncoder": TransformerEncoder}
    )

#print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")
"""

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:915: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:915: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:915: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


25/25 ━━━━━━━━━━━━━━━━━━━━ 14s 223ms/step - accuracy: 0.1135 - loss: 5.2722 - val_accuracy: 0.1550 - val_loss: 2.2174
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 87ms/step - accuracy: 0.1602 - loss: 2.7760 - val_accuracy: 0.4800 - val_loss: 1.4796
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - accuracy: 0.3671 - loss: 1.9746 - val_accuracy: 0.6800 - val_loss: 0.8790
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step - accuracy: 0.6802 - loss: 0.9185 - val_accuracy: 0.7900 - val_loss: 0.6186
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - accuracy: 0.8344 - loss: 0.4789 - val_accuracy: 0.5900 - val_loss: 1.3309
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - accuracy: 0.8892 - loss: 0.3203 - val_accuracy: 0.8500 - val_loss: 0.4216
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.9283 - loss: 0.2082 - val_accuracy: 0.9000 - val_loss: 0.3286
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 79ms/step - accuracy: 0.9611 - loss: 0.1251 - val_accuracy: 0.7900 - val_loss:

'\nmodel = keras.models.load_model(\n    "transformer_encoder.h5",\n    custom_objects={"TransformerEncoder": TransformerEncoder}\n    )\n\n#print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")\n'